In [2]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import random
import seaborn as sns

In [3]:
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif'] 

rc_pub={'font.size': 20, 'axes.labelsize': 20, 'legend.fontsize': 20, 
        'axes.titlesize': 25, 'xtick.labelsize': 20, 'ytick.labelsize': 20, 
        'axes.linewidth':1.5, 'lines.linewidth': 2.0,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}

# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(['cyan', 'magenta','gray','crimson','purple'], 5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc=rc_pub)
    sns.set_style("ticks", {"xtick.major.size": 5, "ytick.major.size": 5})

# to restore the defaults, call plt.rcdefaults() 
set_pub_plots()

# Load the data

In [ ]:
# v1hpc_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1LGN_spks_CQuinn.pkl")
# v1hpc_psth = pd.read_hdf(r"U:\Papers\D&P Osc in Mouse Vis Sys\Data Analysis\Units\V1_LGN\V1_LGN_PSTH_Final.hdf5")

# v1hpc_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1LP_spks_CQuinn.pkl")
# v1hpc_psth = pd.read_hdf('u:/Papers/D&P Osc in Mouse Vis Sys/Data Analysis/Units/V1_LP/LP_PSTH_remLT_300sps_remGT-0o10ampWF_LP_GUI.hdf5')

# v1hpc_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1RSC_spks_CQuinn.pkl")
# v1hpc_psth = pd.read_hdf('u:/Papers/D&P Osc in Mouse Vis Sys/Data Analysis/Units/V1_RSC/V1_RSC_2kmeans_PSTH_GUI_mapped_sliced.hdf5')

v1hpc_spk = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\V1HPC_spks_CQuinn.pkl")
v1hpc_osc = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\V1HPC_cycling_units.pkl")
# v1hpc_psth = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1HPC_psth.pkl")

v1hpc_spk.head()

In [ ]:
v1_spk = v1hpc_spk[(v1hpc_spk.region == 'v1') & (v1hpc_spk.stim == 'post')]    #cut to only V1 units post training
v1osc_spk = v1_spk[v1_spk['cuid'].isin(v1hpc_osc['V1'])]                       #dataframe of only oscillator units
v1nonosc_spk = v1_spk[~v1_spk['cuid'].isin(v1hpc_osc['V1'])]                   #dataframe of non-oscillator units

print(v1_spk.cuid.nunique(), v1osc_spk.cuid.nunique(), v1nonosc_spk.cuid.nunique())

---

In [283]:
def autocorr(x, lags):
    xcorr = np.correlate(x - x.mean(), x - x.mean(), 'full')     # Compute the autocorrelation
    xcorr = xcorr[xcorr.size//2:] / xcorr.max()               # Convert to correlation coefficients
    return xcorr[:lags+1]                                     # Return only requested lags

In [ ]:
num_trials = 15

for unit in v1_spk.cuid.unique():
    #extract out the spike times for the unit randomly chosen above
    osc_unit = v1_spk[(v1_spk.cuid == unit) & (v1_spk.trial < num_trials)]
    osc_unit_spk = osc_unit.trial_spikes.values
    # osc_unit_spk = osc_unit.spikes.values
    
    #plot the spike times
    fig, ax = plt.subplots(1,2, figsize=(12,2))
    ax[0].set_title(unit)
    ax[0].plot(osc_unit_spk, np.zeros(len(osc_unit_spk)), 'k.')
    ax[0].set_xlabel('Tims (s)')
    ax[0].set_yticks(ticks=[-1,0,1])
    
    # perform autocorrelation
    time_bins = np.arange(0, 2.9, 0.05)                         # Define the time bins
    Increments, _ = np.histogram(osc_unit_spk, time_bins)   # ... and compute a histogram of the data
    ACF = autocorr(Increments, 100)
    
    #plot the autocorrelation
    ax[1].plot(ACF, 'k',)                  # Plot autocorrelation vs lags,
    # plt.xlim([0, 100])                 # ... set x-limits
    # plt.ylim([-.2, .2])                # ... and y-limits
    ax[1].set_xlabel('Lag (ms)')                # ... with axes labeled
    ax[1].set_ylabel('Autocorrelation')
    ax[1].set_xticks(ticks=[0,20,40,60], labels=[0, int(20/0.05), int(40/0.05), int(60/0.05)])
    sns.despine()
    plt.show()

Good units (maybe oscillating?)
- et20_390
- et20_397
- et323_440
- et3832_392
- et3232_452

OSCILLATORS
- et30_442
- et30_418

NON OSCILLATORS
- et10_364

---

In [ ]:
# THESE ARE TWO GOOD OSCILLATOR UNITS
unit_id = 'et30_442'
unit_id = 'et30_452'
unit_id = 'et30_418'

# THESE ARE TWO NON-OSCILLATOR UNITS
unit_id = 'et2_471'
unit_id = 'et30_364'
unit_id = 'et3_361'
unit_id = 'et3_379'
unit_id = 'et3_391'

num_trials = 20

#extract out the spike times for the unit randomly chosen above
osc_unit = v1_spk[(v1_spk.cuid == unit_id) & (v1_spk.trial < num_trials)]
osc_unit_spk = osc_unit.trial_spikes.values

# perform autocorrelation
time_bins = np.arange(0, 2.9, 0.05)                         # Define the time bins
Increments, _ = np.histogram(osc_unit_spk, time_bins)   # ... and compute a histogram of the data
ACF = autocorr(Increments, 100)

#plot the autocorrelation
plt.axhline(0, color='grey', linestyle='--')
plt.plot(ACF, 'k-')                   # Plot autocorrelation vs lags,
plt.xlabel('Lag (ms)')                # ... with axes labeled
plt.ylabel('Autocorrelation')
plt.xticks(ticks=[0,20,40,60], labels=[0, int(20/0.05), int(40/0.05), int(60/0.05)])
plt.yticks(ticks=[-0.3,0,0.3,0.6])
plt.xlim([0,40])
plt.ylim([-0.3,0.65])
sns.despine()
# Change the end of this next line with a new file name!!
plt.savefig(r"U:\Papers\D&P Osc in Mouse Vis Sys\New Submission\Figures\nonosc6_autoCorrelogram.pdf", transparent=True)
plt.show()